In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import os

from PIL import Image 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm

In [ ]:
from tensorflow.keras.applications import  VGG16,EfficientNetB7,MobileNetV2,Xception,ResNet152V2,NasNetLarge
from tensorflow.keras.layers import Conv2D 
from tensorflow.keras.layers import Dense,Flatten,Dropout,AveragePooling2D,MaxPool2D,Input
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Model
from tensorflow.keras.layers  import Concatenate
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm
%load_ext tensorboard
import datetime

In [ ]:
batch_size=32

training_datagenarator= ImageDataGenerator(rescale=1./255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest",validation_split=0.1)

In [ ]:
train=training_datagenarator.flow_from_directory('/content/drive/MyDrive/train_images',
                                                target_size=(224, 224),color_mode='rgb',
                                       class_mode='binary', batch_size=batch_size,subset='training')

validation=training_datagenarator.flow_from_directory('/content/drive/MyDrive/train_images',
                                                target_size=(224, 224),color_mode='rgb',
                                       class_mode='binary', batch_size=batch_size,subset='validation')

Found 2924 images belonging to 2 classes.
Found 324 images belonging to 2 classes.


#simple cnn

In [ ]:
cnn=tf.keras.models.Sequential()

# adding first layer
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',input_shape=[224,224,3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

# adding second layer
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

# adding third layer
cnn.add(tf.keras.layers.Conv2D(filters=256,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

# Flattening
cnn.add(tf.keras.layers.Flatten())

# Fully connected layer
cnn.add(tf.keras.layers.Dense(units=128,activation='relu'))

# Output layers
cnn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [ ]:
cnn.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

filepath="/content/drive/MyDrive/models/savedmodel/simple_cnn_new.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy',  verbose=1, save_best_only=True, mode='auto')

H = cnn.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=20,callbacks=[tensorboard_callback,checkpoint])

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
 1/91 [..............................] - ETA: 18:10 - loss: 0.6940 - accuracy: 0.4375

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


91/91 [==============================] - 1022s 11s/step - loss: 0.7424 - accuracy: 0.6183 - val_loss: 0.5706 - val_accuracy: 0.7000

Epoch 00001: val_accuracy improved from -inf to 0.70000, saving model to /content/drive/MyDrive/models/savedmodel/simple_cnn_new.hdf5
Epoch 2/25
91/91 [==============================] - 141s 2s/step - loss: 0.5529 - accuracy: 0.7216 - val_loss: 0.6939 - val_accuracy: 0.6625

Epoch 00002: val_accuracy did not improve from 0.70000
Epoch 3/25
91/91 [==============================] - 137s 1s/step - loss: 0.5398 - accuracy: 0.7196 - val_loss: 0.6037 - val_accuracy: 0.6938

Epoch 00003: val_accuracy did not improve from 0.70000
Epoch 4/25
91/91 [==============================] - 138s 2s/step - loss: 0.5345 - accuracy: 0.7331 - val_loss: 0.5936 - val_accuracy: 0.6938

Epoch 00004: val_accuracy did not improve from 0.70000
Epoch 5/25
91/91 [==============================] - 141s 2s/step - loss: 0.5254 - accuracy: 0.7407 - val_loss: 0.5577 - val_accuracy: 0.7188



#NASNET

In [ ]:
baseModel = tf.keras.applications.NASNetLarge(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

for layer in baseModel.layers:
    layer.trainable = False

In [ ]:

headModel = baseModel.output

headModel = Flatten(name="flatten")(headModel)
headModel = Dense(1024, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(1, activation="sigmoid")(headModel)
net = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
net.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

filepath="/content/drive/MyDrive/models/saved_models/net_new_current.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy',  verbose=1, save_best_only=True, mode='auto')

net.fit_generator(train,validation_data=validation,epochs=20,
                  steps_per_epoch=train.samples//batch_size,
                  validation_steps=validation.samples//batch_size,callbacks=checkpoint)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
54/91 [================>.............] - ETA: 1:02 - loss: 10.6400 - accuracy: 0.7396

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


91/91 [==============================] - 191s 2s/step - loss: 7.5851 - accuracy: 0.7503 - val_loss: 2.9113 - val_accuracy: 0.6594

Epoch 00001: val_accuracy improved from -inf to 0.65938, saving model to /content/drive/MyDrive/models/saved_models/net_new_current.hdf5


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/20
91/91 [==============================] - 222s 2s/step - loss: 0.7041 - accuracy: 0.7977 - val_loss: 0.8206 - val_accuracy: 0.7344

Epoch 00002: val_accuracy improved from 0.65938 to 0.73438, saving model to /content/drive/MyDrive/models/saved_models/net_new_current.hdf5
Epoch 3/20
91/91 [==============================] - 196s 2s/step - loss: 0.5099 - accuracy: 0.8195 - val_loss: 0.9176 - val_accuracy: 0.7219

Epoch 00003: val_accuracy did not improve from 0.73438
Epoch 4/20
91/91 [==============================] - 166s 2s/step - loss: 0.4805 - accuracy: 0.8326 - val_loss: 0.7840 - val_accuracy: 0.7031

Epoch 00004: val_accuracy did not improve from 0.73438
Epoch 5/20
91/91 [==============================] - 171s 2s/step - loss: 0.4604 - accuracy: 0.8385 - val_loss: 0.8283 - val_accuracy: 0.7312

Epoch 00005: val_accuracy did not improve from 0.73438
Epoch 6/20
91/91 [==============================] - 167s 2s/step - loss: 0.4114 - accuracy: 0.8465 - val_loss: 0.7486 - val_acc

#MOBILENET

In [ ]:
baseModel = tf.keras.applications.MobileNetV2(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

for layer in baseModel.layers:
    layer.trainable = False

9420800/9406464 [==============================] - 0s 0us/step


In [ ]:

headModel = baseModel.output

headModel = Flatten(name="flatten")(headModel)
headModel = Dense(1024, activation="relu")(headModel)
#headModel = Dropout(0.5)(headModel)
headModel = Dense(1, activation="sigmoid")(headModel)
net = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
net.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

filepath="/content/drive/MyDrive/models/saved_models/mobilenet_new_current.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy',  verbose=1, save_best_only=True, mode='auto')

net.fit_generator(train,validation_data=validation,epochs=20,
                  steps_per_epoch=train.samples//batch_size,
                  validation_steps=validation.samples//batch_size,callbacks=checkpoint)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
51/91 [===============>..............] - ETA: 56s - loss: 12.2738 - accuracy: 0.6979

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


91/91 [==============================] - 144s 2s/step - loss: 7.8554 - accuracy: 0.7151 - val_loss: 1.1874 - val_accuracy: 0.6719

Epoch 00001: val_accuracy improved from -inf to 0.67188, saving model to /content/drive/MyDrive/models/saved_models/mobilenet_new_current.hdf5


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/20
91/91 [==============================] - 137s 2s/step - loss: 0.4414 - accuracy: 0.8081 - val_loss: 0.6952 - val_accuracy: 0.7063

Epoch 00002: val_accuracy improved from 0.67188 to 0.70625, saving model to /content/drive/MyDrive/models/saved_models/mobilenet_new_current.hdf5
Epoch 3/20
91/91 [==============================] - 133s 1s/step - loss: 0.4091 - accuracy: 0.8271 - val_loss: 0.6754 - val_accuracy: 0.7125

Epoch 00003: val_accuracy improved from 0.70625 to 0.71250, saving model to /content/drive/MyDrive/models/saved_models/mobilenet_new_current.hdf5
Epoch 4/20
91/91 [==============================] - 136s 1s/step - loss: 0.3894 - accuracy: 0.8416 - val_loss: 0.6434 - val_accuracy: 0.6875

Epoch 00004: val_accuracy did not improve from 0.71250
Epoch 5/20
91/91 [==============================] - 135s 1s/step - loss: 0.3932 - accuracy: 0.8309 - val_loss: 0.6070 - val_accuracy: 0.7281

Epoch 00005: val_accuracy improved from 0.71250 to 0.72812, saving model to /content/

#efficent

In [ ]:
baseModel = tf.keras.applications.EfficientNetB0(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

for layer in baseModel.layers:
    layer.trainable = False

16719872/16705208 [==============================] - 0s 0us/step


In [ ]:

headModel = baseModel.output
headModel       = Conv2D(256,(3,3),padding='same',activation='relu')(headModel )
headModel       = MaxPool2D(pool_size=(4,4),padding='same')(headModel )
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(1, activation="sigmoid")(headModel)
net1 = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
net1.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

filepath="/content/drive/MyDrive/models/saved_models/effiecent.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy',  verbose=1, save_best_only=True, mode='auto')

net1.fit_generator(train,validation_data=validation,epochs=20,
                  steps_per_epoch=train.samples//batch_size,
                  validation_steps=validation.samples//batch_size,callbacks=checkpoint)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
41/91 [============>.................] - ETA: 13:47 - loss: 1.5409 - accuracy: 0.5610

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


91/91 [==============================] - 1614s 18s/step - loss: 1.0718 - accuracy: 0.5813 - val_loss: 0.6764 - val_accuracy: 0.6062

Epoch 00001: val_accuracy improved from -inf to 0.60625, saving model to /content/drive/MyDrive/models/saved_models/effiecent.hdf5


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/20
91/91 [==============================] - 140s 2s/step - loss: 0.6902 - accuracy: 0.6093 - val_loss: 0.6870 - val_accuracy: 0.6062

Epoch 00002: val_accuracy did not improve from 0.60625
Epoch 3/20
91/91 [==============================] - 139s 2s/step - loss: 0.6845 - accuracy: 0.6079 - val_loss: 0.6816 - val_accuracy: 0.6125

Epoch 00003: val_accuracy improved from 0.60625 to 0.61250, saving model to /content/drive/MyDrive/models/saved_models/effiecent.hdf5
Epoch 4/20
91/91 [==============================] - 139s 2s/step - loss: 0.6804 - accuracy: 0.6072 - val_loss: 0.6789 - val_accuracy: 0.6062

Epoch 00004: val_accuracy did not improve from 0.61250
Epoch 5/20
91/91 [==============================] - 139s 2s/step - loss: 0.6769 - accuracy: 0.6089 - val_loss: 0.6761 - val_accuracy: 0.6062

Epoch 00005: val_accuracy did not improve from 0.61250
Epoch 6/20
91/91 [==============================] - 139s 2s/step - loss: 0.6748 - accuracy: 0.6082 - val_loss: 0.6735 - val_accuracy:

#efficent

In [ ]:
baseModel = tf.keras.applications.EfficientNetB0(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

for layer in baseModel.layers:
    layer.trainable = False

16719872/16705208 [==============================] - 0s 0us/step


In [ ]:

headModel = baseModel.output
headModel       = Conv2D(256,(3,3),padding='same',activation='relu')(headModel )
headModel       = MaxPool2D(pool_size=(4,4),padding='same')(headModel )
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(1, activation="sigmoid")(headModel)
net1 = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
net1.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

filepath="/content/drive/MyDrive/models/saved_models/effiecent.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy',  verbose=1, save_best_only=True, mode='auto')

net1.fit_generator(train,validation_data=validation,epochs=20,
                  steps_per_epoch=train.samples//batch_size,
                  validation_steps=validation.samples//batch_size,callbacks=checkpoint)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
41/91 [============>.................] - ETA: 13:47 - loss: 1.5409 - accuracy: 0.5610

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


91/91 [==============================] - 1614s 18s/step - loss: 1.0718 - accuracy: 0.5813 - val_loss: 0.6764 - val_accuracy: 0.6062

Epoch 00001: val_accuracy improved from -inf to 0.60625, saving model to /content/drive/MyDrive/models/saved_models/effiecent.hdf5


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/20
91/91 [==============================] - 140s 2s/step - loss: 0.6902 - accuracy: 0.6093 - val_loss: 0.6870 - val_accuracy: 0.6062

Epoch 00002: val_accuracy did not improve from 0.60625
Epoch 3/20
91/91 [==============================] - 139s 2s/step - loss: 0.6845 - accuracy: 0.6079 - val_loss: 0.6816 - val_accuracy: 0.6125

Epoch 00003: val_accuracy improved from 0.60625 to 0.61250, saving model to /content/drive/MyDrive/models/saved_models/effiecent.hdf5
Epoch 4/20
91/91 [==============================] - 139s 2s/step - loss: 0.6804 - accuracy: 0.6072 - val_loss: 0.6789 - val_accuracy: 0.6062

Epoch 00004: val_accuracy did not improve from 0.61250
Epoch 5/20
91/91 [==============================] - 139s 2s/step - loss: 0.6769 - accuracy: 0.6089 - val_loss: 0.6761 - val_accuracy: 0.6062

Epoch 00005: val_accuracy did not improve from 0.61250
Epoch 6/20
91/91 [==============================] - 139s 2s/step - loss: 0.6748 - accuracy: 0.6082 - val_loss: 0.6735 - val_accuracy:

#vggnet16

In [ ]:
baseModel = tf.keras.applications.VGG16(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

for layer in baseModel.layers:
    layer.trainable = False

In [ ]:

headModel = baseModel.output
headModel       = Conv2D(256,(4,4),padding='same',activation='relu')(headModel )
headModel       = MaxPool2D(pool_size=(6,6))(headModel )
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(1, activation="sigmoid")(headModel)
net3 = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
net3.fit_generator(train,validation_data=validation,epochs=10,
                  steps_per_epoch=train.samples//batch_size,
                  validation_steps=validation.samples//batch_size,callbacks=checkpoint)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
59/91 [==================>...........] - ETA: 47s - loss: 0.1803 - accuracy: 0.9208

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


91/91 [==============================] - 147s 2s/step - loss: 0.1771 - accuracy: 0.9274 - val_loss: 1.0028 - val_accuracy: 0.7500

Epoch 00001: val_accuracy did not improve from 0.78125
Epoch 2/10
91/91 [==============================] - 147s 2s/step - loss: 0.1578 - accuracy: 0.9346 - val_loss: 0.9315 - val_accuracy: 0.7437

Epoch 00002: val_accuracy did not improve from 0.78125
Epoch 3/10
91/91 [==============================] - 146s 2s/step - loss: 0.1857 - accuracy: 0.9194 - val_loss: 0.9430 - val_accuracy: 0.7094

Epoch 00003: val_accuracy did not improve from 0.78125
Epoch 4/10
91/91 [==============================] - 146s 2s/step - loss: 0.1823 - accuracy: 0.9312 - val_loss: 0.6514 - val_accuracy: 0.7844

Epoch 00004: val_accuracy improved from 0.78125 to 0.78438, saving model to /content/drive/MyDrive/models/saved_models/vggnet16_img1.hdf5
Epoch 5/10
91/91 [==============================] - 147s 2s/step - loss: 0.1606 - accuracy: 0.9357 - val_loss: 1.2027 - val_accuracy: 0.7094

#VGG19

In [ ]:
baseModel = tf.keras.applications.VGG19(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

for layer in baseModel.layers:
    layer.trainable = False

80150528/80134624 [==============================] - 1s 0us/step


In [ ]:

headModel = baseModel.output
headModel       = Conv2D(256,(4,4),padding='same',activation='relu')(headModel )
headModel       = MaxPool2D(pool_size=(6,6))(headModel )
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(1, activation="sigmoid")(headModel)
net4 = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
net4.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

filepath="/content/drive/MyDrive/models/saved_models/vggnet19_img.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy',  verbose=1, save_best_only=True, mode='auto')

net4.fit_generator(train,validation_data=validation,epochs=20,
                  steps_per_epoch=train.samples//batch_size,
                  validation_steps=validation.samples//batch_size,callbacks=checkpoint)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
68/91 [=====================>........] - ETA: 35s - loss: 0.7241 - accuracy: 0.6071

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


91/91 [==============================] - 151s 2s/step - loss: 0.6870 - accuracy: 0.6366 - val_loss: 0.6680 - val_accuracy: 0.6812

Epoch 00001: val_accuracy improved from -inf to 0.68125, saving model to /content/drive/MyDrive/models/saved_models/vggnet19_img.hdf5
Epoch 2/20
91/91 [==============================] - 150s 2s/step - loss: 0.5260 - accuracy: 0.7552 - val_loss: 0.5793 - val_accuracy: 0.7031

Epoch 00002: val_accuracy improved from 0.68125 to 0.70312, saving model to /content/drive/MyDrive/models/saved_models/vggnet19_img.hdf5
Epoch 3/20
91/91 [==============================] - 150s 2s/step - loss: 0.4847 - accuracy: 0.7857 - val_loss: 0.5548 - val_accuracy: 0.7000

Epoch 00003: val_accuracy did not improve from 0.70312
Epoch 4/20
91/91 [==============================] - 150s 2s/step - loss: 0.4624 - accuracy: 0.8022 - val_loss: 0.5851 - val_accuracy: 0.7031

Epoch 00004: val_accuracy did not improve from 0.70312
Epoch 5/20
91/91 [==============================] - 150s 2s/st

#

In [1]:
from prettytable import PrettyTable 
  
# Specify the Column Names while initializing the Table 
myTable = PrettyTable(["pretrained model", " validation accuracy ","size"]) 
  
# Add rows 
myTable.add_row(["simplecnn", "0.7406","296mb"]) 
myTable.add_row(["Mobilenetv2", "0.7781","744mb"]) 
myTable.add_row(["vggnet16", "0.7843","80mb"]) 
myTable.add_row(["vggnet19", "0.7500","265mb"]) 
myTable.add_row(["nasnetlarge", "0.7406","80.74mb"]) 
myTable.add_row(["efficientnetb0", "0.6125","51mb"]) 


print(myTable)


+------------------+-----------------------+---------+
| pretrained model |  validation accuracy  |   size  |
+------------------+-----------------------+---------+
|    simplecnn     |         0.7406        |  296mb  |
|   Mobilenetv2    |         0.7781        |  744mb  |
|     vggnet16     |         0.7843        |   80mb  |
|     vggnet19     |         0.7500        |  265mb  |
|   nasnetlarge    |         0.7406        | 80.74mb |
|  efficientnetb0  |         0.6125        |   51mb  |
+------------------+-----------------------+---------+


#checking on test data

#simple cnn

In [ ]:
simple_cnn=load_model('/content/drive/MyDrive/current_working_models/simple_cnn_new.hdf5')

y_pred=y_prediction(all_paths,simple_cnn)
score1=accuracy_score(labels,y_pred)
print('accuracy of simple cnn',score1)

accuracy of simple cnn 0.6363636363636364


#VGGNET16

In [ ]:
vgg16=load_model('/content/drive/MyDrive/current_working_models/vggnet16_img1.hdf5')

y_pred=y_prediction(all_paths,vgg16)
score2=accuracy_score(labels,y_pred)
print('accuracy of vggnet16',score2)

accuracy of vggnet16 0.9090909090909091


#*VGGNET19*

In [ ]:
vgg19=load_model('/content/drive/MyDrive/current_working_models/vggnet19_img.hdf5')

y_pred=y_prediction(all_paths,vgg19)
score3=accuracy_score(labels,y_pred)
print('accuracy of VGGNET19',score3)

accuracy of VGGNET19 0.7727272727272727


#NASNET

In [ ]:
naslarge=load_model('/content/drive/MyDrive/current_working_models/net_new_current.hdf5')

y_pred=y_prediction(all_paths,naslarge)
score4=accuracy_score(labels,y_pred)
print('accuracy of Nasnet Large',score4)

accuracy of Nasnet Large 0.7727272727272727


#MOBILENET

In [ ]:
mobilenet=load_model('/content/drive/MyDrive/current_working_models/mobilenet_new_current.hdf5')

y_pred=y_prediction(all_paths,mobilenet)
score5=accuracy_score(labels,y_pred)
print('accuracy of Mobilenet',score5)

accuracy of Mobilenet 0.7272727272727273


#EFFIECENTNET

In [ ]:
efficent=load_model('/content/drive/MyDrive/current_working_models/effiecent.hdf5')

y_pred=y_prediction(all_paths,efficent)
score6=accuracy_score(labels,y_pred)
print('accuracy of EfficientNet',score6)

accuracy of EfficientNet 0.5454545454545454


#RESNET

In [ ]:
resnet=load_model('/content/drive/MyDrive/current_working_models/resnetv2.hdf5')

y_pred=y_prediction(all_paths,resnet)
score7=accuracy_score(labels,y_pred)
print('accuracy of RESNET',score7)

accuracy of RESNET 0.8181818181818182


#therefore we got .90 accuaracy on gtest data for VGGNET16